In [2]:
import rainflow
from numpy import fabs as fabs

import pickle
import numpy as np
import itertools
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objs as go

from pyts.image import GramianAngularField
import matplotlib.pyplot as plt
import scipy

import lzma
import os
import re
import json

import sys
sys.path.insert(1, '/home/sph3re/Programming/windturbine/src/env')
import hparams
from __future__ import division
from collections import deque, defaultdict
import math
from helpers import *

In [3]:

def rainflow_nrel(array_ext,
             flm=0, l_ult=1e16, uc_mult=0.5):
    """ Rainflow counting of a signal's turning points with Goodman correction
    
        Args:
            array_ext (numpy.ndarray): array of turning points
        
        Keyword Args:
            flm (float): fixed-load mean [opt, default=0]
            l_ult (float): ultimate load [opt, default=1e16]
            uc_mult (float): partial-load scaling [opt, default=0.5]
            
        Returns:
            array_out (numpy.ndarray): (5 x n_cycle) array of rainflow values:
                                        1) load range
                                        2) range mean
                                        3) Goodman-adjusted range
                                        4) cycle count
                                        5) Goodman-adjusted range with flm = 0
            
    """
    
    flmargin = l_ult - fabs(flm)            # fixed load margin
    tot_num = array_ext.size                # total size of input array
    array_out = np.zeros((5, tot_num-1))    # initialize output array
    
    pr = 0                                  # index of input array
    po = 0                                  # index of output array
    j = -1                                  # index of temporary array "a"
    a  = np.empty(array_ext.shape)          # temporary array for algorithm
    
    # loop through each turning point stored in input array
    for i in range(tot_num):
        
        j += 1                  # increment "a" counter
        a[j] = array_ext[pr]    # put turning point into temporary array
        pr += 1                 # increment input array pointer
        
        while ((j >= 2) & (fabs( a[j-1] - a[j-2] ) <= \
                fabs( a[j] - a[j-1]) ) ):
            lrange = fabs( a[j-1] - a[j-2] )
              
            # partial range
            if j == 2:
                mean      = ( a[0] + a[1] ) / 2.
                adj_range = lrange * flmargin / ( l_ult - fabs(mean) )
                adj_zero_mean_range = lrange * l_ult / ( l_ult - fabs(mean) )
                a[0]=a[1]
                a[1]=a[2]
                j=1
                if (lrange > 0):
                    array_out[0,po] = lrange
                    array_out[1,po] = mean
                    array_out[2,po] = adj_range
                    array_out[3,po] = uc_mult
                    array_out[4,po] = adj_zero_mean_range
                    po += 1
                
            # full range
            else:
                mean      = ( a[j-1] + a[j-2] ) / 2.
                adj_range = lrange * flmargin / ( l_ult - fabs(mean) )
                adj_zero_mean_range = lrange * l_ult / ( l_ult - fabs(mean) )
                a[j-2]=a[j]
                j=j-2
                if (lrange > 0):
                    array_out[0,po] = lrange
                    array_out[1,po] = mean
                    array_out[2,po] = adj_range
                    array_out[3,po] = 1.00
                    array_out[4,po] = adj_zero_mean_range
                    po += 1
                    
    # partial range
    for i in range(j):
        lrange    = fabs( a[i] - a[i+1] );
        mean      = ( a[i] + a[i+1] ) / 2.
        adj_range = lrange * flmargin / ( l_ult - fabs(mean) )
        adj_zero_mean_range = lrange * l_ult / ( l_ult - fabs(mean) )
        if (lrange > 0):
            array_out[0,po] = lrange
            array_out[1,po] = mean
            array_out[2,po] = adj_range
            array_out[3,po] = uc_mult
            array_out[4,po] = adj_zero_mean_range
            po += 1  
            
    # get rid of unused entries
    array_out = array_out[:,:po]

    return array_out

def turningpoints(lst):
    dx = np.diff(lst)
    return np.sum(dx[1:] * dx[:-1] < 0)

In [4]:
def count_cycles(series, ndigits=None, nbins=None, binsize=None):
    """Count cycles in the series.
    Parameters
    ----------
    series : iterable sequence of numbers
    ndigits : int, optional
        Round cycle magnitudes to the given number of digits before counting.
        Use a negative value to round to tens, hundreds, etc.
    nbins : int, optional
        Specifies the number of cycle-counting bins.
    binsize : int, optional
        Specifies the width of each cycle-counting bin
    Arguments ndigits, nbins and binsize are mutually exclusive.
    Returns
    -------
    A sorted list containing pairs of range and cycle count.
    The counts may not be whole numbers because the rainflow counting
    algorithm may produce half-cycles. If binning is used then ranges
    correspond to the right (high) edge of a bin.
    """
    if sum(value is not None for value in (ndigits, nbins, binsize)) > 1:
        raise ValueError(
            "Arguments ndigits, nbins and binsize are mutually exclusive"
        )

    counts = defaultdict(float)
    cycles = (
        (rng, count)
        for rng, mean, count, i_start, i_end in rainflow.extract_cycles(series)
    )
    

    if nbins is not None:
        binsize = (max(series) - min(series)) / nbins

    if binsize is not None:
        nmax = 0
        for rng, count in cycles:
            if count == 0.5:
                rng *= 0.5
            n = int(math.ceil(rng / binsize))  # using int for Python 2 compatibility
            counts[n * binsize] += count
            nmax = max(n, nmax)

        for i in range(1, nmax):
            counts.setdefault(i * binsize, 0.0)

    elif ndigits is not None:
        round_ = _get_round_function(ndigits)
        for rng, count in cycles:
            counts[round_(rng)] += count

    else:
        for rng, count in cycles:
            counts[rng] += count

    return sorted(counts.items())

In [52]:
def calc_something_like_del(signal, dt=0.1, fdel=1, m=1, texp=3):
    rainflow_data = np.array(list(rainflow.extract_cycles(signal)))

    lr = rainflow_data[:,0]
#     lm = rainflow_data[:,1]
    n = rainflow_data[:,2]

    # lrf = lr * ((lult - np.abs(lmf)) / (lult - np.abs(lm)))
    lrf = lr
    
    T = len(signal)*dt
    nsteq = fdel*T**texp

    delst = np.sum(np.sum(n * lrf ** m)/nsteq) ** (1/m)
    return delst

In [6]:
def calc_del(signal, dt=0.1, fdel=1, m=1):

    rainflow_data = np.array(list(rainflow.extract_cycles(signal)))

    lr = rainflow_data[:,0]
#     lm = rainflow_data[:,1]
    n = rainflow_data[:,2]

    # lrf = lr * ((lult - np.abs(lmf)) / (lult - np.abs(lm)))
    lrf = lr

    T = len(signal)*dt
    nsteq = fdel*T

    delst = (np.sum(np.sum(n * lrf ** m)) / nsteq) ** (1/m)
    return delst


In [7]:
def calc_del_pd(df, dt=0.1, fdel=1, m=1):
    lr = np.array(df['Range'])
    #     lm = rainflow_data[:,1]
    n = np.array(df['Count'])

    # lrf = lr * ((lult - np.abs(lmf)) / (lult - np.abs(lm)))
    lrf = lr

    T = len(df)*dt
    nsteq = fdel*T

    delst = (np.sum(np.sum(n * lrf ** m)) / np.sum(nsteq)) ** (1/m)
    return delst

In [5]:
with lzma.open('/home/sph3re/Programming/windturbine/results/sac-turbulent-1/data_15/eval_630.xz', 'rb') as f:
    data = pickle.load(f)
parsed = parse_data(1, data, True)

2021-11-27 21:43:27.136941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/sph3re/.mujoco/mujoco200/bin
2021-11-27 21:43:27.136966: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/sph3re/.anaconda3/envs/rl/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [34]:
!ls /home/sph3re/Downloads/DataCrunch/Data

NTM_ws_20ms_d8_s114_new.rcc  NTM_ws_24ms_d8_s114_new.rcc
NTM_ws_20ms_d8_s114_new.txt  NTM_ws_24ms_d8_s114_new.txt
NTM_ws_21ms_d8_s114_new.rcc  NTM_ws_25ms_d8_s114_new.rcc
NTM_ws_21ms_d8_s114_new.txt  NTM_ws_25ms_d8_s114_new.txt
NTM_ws_22ms_d8_s114_new.rcc  NTM_ws_26ms_d8_s114_new.rcc
NTM_ws_22ms_d8_s114_new.txt  NTM_ws_26ms_d8_s114_new.txt
NTM_ws_23ms_d8_s114_new.rcc  NTM_ws_27ms_d8_s114_new.rcc
NTM_ws_23ms_d8_s114_new.txt  NTM_ws_27ms_d8_s114_new.txt


In [140]:
filename = '/home/sph3re/Downloads/DataCrunch/Data/NTM_ws_20ms_d8_s114_new'
ref_signal = pd.read_csv('%s.txt' % filename, header=[1], sep='\t', skipinitialspace=True)
signal = np.array(ref_signal['Out Of Plane Root Bending Moment Blade 1 [Nm]']).flatten()

In [156]:
ref_results = pd.read_csv('%s.rcc' % filename, header=[0,1], sep='\t', skipinitialspace=True)
ref_results['Range'] = ref_results['Oo_PlaneBM[X]']*1000
ref_results['Count'] = ref_results['Oo_PlaneBM[Y]']

{
    'del': calc_del_pd(ref_results, m=10),
    'len': len(ref_results),
    'sum_count': sum(ref_results['Count']),
    'sum_range': sum(ref_results['Range'])
}

{'del': 40181844.95957512,
 'len': 128,
 'sum_count': 2549.640299999998,
 'sum_range': 3354823100.0}

In [145]:
ref_results = pd.read_csv('/home/sph3re/Downloads/CrunchData-NoBins/NTM_ws_20ms_d8_s114_new_Oo_PlaneBM.rcc', header=[0], sep='    ', skipinitialspace=True)
ref_results['Range'] = ref_results['Ranges']*1000
ref_results['Count'] = ref_results['NCycles']
ref_results['orig_idx'] = ref_results.index
ref_results.sort_values('Range', inplace=True, ignore_index=True)

{
    'del': calc_del_pd(ref_results, m=10),
    'len': len(ref_results),
    'sum_count': sum(ref_results['Count']),
    'sum_range': sum(ref_results['Range'])
}

{'del': 29857556.467972513,
 'len': 2556,
 'sum_count': 2552.0,
 'sum_range': 3456378046.0}

In [146]:
python_res = pd.DataFrame(count_cycles(signal), columns=['Range', 'Count'])
{
    'del': calc_del_pd(python_res, m=10),
    'len': len(python_res),
    'sum_count': sum(python_res['Count']),
    'sum_range': sum(python_res['Range'])
}

{'del': 30064777.037544508,
 'len': 2386,
 'sum_count': 2552.0,
 'sum_range': 3401549122.0}

In [154]:
python_res = pd.DataFrame(count_cycles(signal, nbins=128), columns=['Range', 'Count'])
{
    'del': calc_del_pd(python_res, m=10),
    'len': len(python_res),
    'sum_count': sum(python_res['Count']),
    'sum_range': sum(python_res['Range'])
}

{'del': 26472366.2053289,
 'len': 70,
 'sum_count': 2552.0,
 'sum_range': 1017667897.1484375}

In [134]:
signal = parsed['ipc_reference']['oop blade root bending moment blade 1 [N]']
signal

0       2.345599e+07
1       2.363081e+07
2       2.356313e+07
3       2.368734e+07
4       2.392789e+07
            ...     
1995    2.726641e+07
1996    2.821087e+07
1997    2.891133e+07
1998    2.938702e+07
1999    2.945762e+07
Name: oop blade root bending moment blade 1 [N], Length: 2000, dtype: float64

In [67]:
python_res = pd.DataFrame(rainflow.extract_cycles(signal), columns=["Range", "Mean", "Count", "Start idx", "End idx"])
python_res.sort_values('Range', inplace=True, ignore_index=True)

In [84]:
dx = np.diff(signal)
tps = np.concatenate([[True], (dx[1:] * dx[:-1] < 0), [True]])
nrel_res = pd.DataFrame(rainflow_nrel(np.array(signal[tps])).transpose(), columns=['Range', 'Count', 'Mean', 'Count', 'GAR-ZFLM'])
nrel_res.sort_values('Range', inplace=True, ignore_index=True)

In [85]:
python_res

,Range,Mean,Count,Start idx,End idx
0,5.444366e+02,2.594351e+07,1.0,1748,1749
1,2.007580e+03,2.139745e+07,1.0,111,112
2,2.772391e+03,1.677356e+07,1.0,1299,1300
3,1.072109e+04,2.298321e+07,1.0,1059,1060
4,1.392687e+04,2.609418e+07,1.0,4,5
...,...,...,...,...,...
602,1.498269e+07,2.216381e+07,0.5,1945,1999
603,2.005192e+07,2.589387e+07,0.5,72,929
604,2.121907e+07,2.528200e+07,0.5,1471,1945
605,2.146959e+07,2.515674e+07,0.5,1266,1471


In [86]:
nrel_res

,Range,Count,Mean,Count,GAR-ZFLM
0,5.444366e+02,2.594351e+07,5.444366e+02,1.0,5.444366e+02
1,2.007580e+03,2.139745e+07,2.007580e+03,1.0,2.007580e+03
2,2.772391e+03,1.677356e+07,2.772391e+03,1.0,2.772391e+03
3,1.072109e+04,2.298321e+07,1.072109e+04,1.0,1.072109e+04
4,1.392687e+04,2.609418e+07,1.392687e+04,1.0,1.392687e+04
...,...,...,...,...,...
602,1.498269e+07,2.216381e+07,1.498269e+07,0.5,1.498269e+07
603,2.005192e+07,2.589387e+07,2.005192e+07,0.5,2.005192e+07
604,2.121907e+07,2.528200e+07,2.121907e+07,0.5,2.121907e+07
605,2.146959e+07,2.515674e+07,2.146959e+07,0.5,2.146959e+07


In [80]:
np.sum(np.abs(nrel_res['Range'] - python_res['Range']))

1087027937.932522

In [73]:
np.sum(np.abs(nrel_res['GAR'] - python_res['Count']))

0.0

In [76]:
np.sum(np.abs(nrel_res['Mean'] - python_res['Mean']))

13393072916.119474

In [94]:
rainflow_data = python_res

In [283]:
signal = np.array(parsed['orig_states']['oop blade root bending moment blade 1 [N]'])
rl = np.array([calc_del(signal, m=m) for m in range(1,14)])
signal = np.array(parsed['ipc_reference']['oop blade root bending moment blade 1 [N]'])
ipc = np.array([calc_del(signal, m=m) for m in range(1,14)])

In [284]:
rl - ipc

array([3085616.43552002,  822415.31814261,    4096.34233904,
       -397402.37407771, -584870.67025954, -683356.63511086,
       -749702.76462159, -803474.53081425, -850660.24805102,
       -893174.21339039, -931830.36095558, -967157.96578255,
       -999598.61567362])

In [211]:
states = garage.np.pad_batch_array(data['episodes'].env_infos['orig_state'], data['episodes'].lengths, 2000)

In [206]:
signal = np.array(parsed['orig_states'][['oop blade root bending moment blade 1 [N]', 'oop blade root bending moment blade 2 [N]']])
signal = signal.transpose()

In [209]:
[calc_agg_del(signal, m=m) for m in range(1,14)]

[6641646.733614378,
 5478464.519877202,
 6212523.324279498,
 7405861.275091234,
 8695871.8402402,
 9898177.653165711,
 10953153.990332045,
 11860972.573497646,
 12640651.028717797,
 13313729.884603642,
 13899279.008851562,
 14412950.325795531,
 14867265.754960615]

In [212]:
parsed['orig_states']

,rotational speed [rad/s],power [W],HH wind velocity [m/s],yaw angle [deg],pitch blade 1 [deg],pitch blade 2 [deg],pitch blade 3 [deg],oop blade root bending moment blade 1 [N],oop blade root bending moment blade 2 [N],oop blade root bending moment blade 3 [N],...,horizontal inflow angle [deg],tower top fore aft acceleration [m/s^2],tower top side side acceleration [m/s^2],tower top X position [m],tower top Y position [m],controller torque [N],controller pitch 1 [deg],controller pitch 2 [deg],controller pitch 3 [deg],azimuthal position of the LSS sum [deg]
0,0.909461,10024.540603,13.963376,8.119111e-15,8.875344,8.075344,8.075344,2.613418e+07,2.340484e+07,1.935956e+07,...,0.0,-0.021137,0.038800,0.232235,-0.095807,1.171176e+07,8.499349,8.499349,8.499349,349.551
1,0.911234,10031.821045,14.142679,-7.392305e-16,9.275344,7.675344,7.859056,2.675718e+07,2.182782e+07,1.830769e+07,...,0.0,-0.062591,-0.016077,0.227717,-0.095291,1.169170e+07,8.522030,8.522030,8.522030,354.767
2,0.912290,10026.248942,14.119777,-8.438564e-14,9.675344,7.275344,8.259056,2.631790e+07,2.144022e+07,1.862539e+07,...,0.0,-0.049836,0.315762,0.222676,-0.093401,1.167428e+07,8.544270,8.544270,8.544270,359.990
3,0.912106,10009.297990,14.002832,-4.439110e-14,10.075344,6.875344,8.659056,2.665502e+07,2.158663e+07,1.940267e+07,...,0.0,-0.040153,-0.059543,0.217160,-0.090390,1.166494e+07,8.567264,8.567264,8.567264,365.217
4,0.911904,9999.071043,13.810796,-4.391899e-15,10.475344,6.475344,8.830463,2.608722e+07,2.281408e+07,2.069246e+07,...,0.0,-0.010053,-0.165995,0.211356,-0.088031,1.166375e+07,8.591797,8.591797,8.591797,370.442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.884522,9985.580132,12.550417,7.419638e-14,7.390502,6.732124,6.258638,2.469846e+07,2.251239e+07,2.044436e+07,...,0.0,-0.030667,-0.106661,0.165826,-0.110419,1.200317e+07,6.729846,6.729846,6.729846,10717.759
1996,0.884497,9979.757271,12.569495,1.921999e-14,7.247674,6.570279,6.445907,2.603793e+07,2.052710e+07,2.192968e+07,...,0.0,0.064629,0.165794,0.171973,-0.109451,1.199961e+07,6.689385,6.689385,6.689385,10722.827
1997,0.884129,9972.648003,12.489997,-1.102013e-14,7.103217,6.560773,6.451890,2.799722e+07,2.079591e+07,2.114565e+07,...,0.0,0.008625,-0.057670,0.178410,-0.108121,1.200100e+07,6.650034,6.650034,6.650034,10727.893
1998,0.883477,9966.452651,12.237620,-1.229980e-14,7.008896,6.323241,6.666931,2.861064e+07,1.980165e+07,2.237760e+07,...,0.0,0.025102,0.118495,0.185066,-0.106336,1.200706e+07,6.612458,6.612458,6.612458,10732.957


In [220]:
calc_agg_del(states[1,:,[7,8,9]], m=10)

13195195.25226475

In [233]:
[np.mean(np.mean(np.abs(np.diff(states[i,:,[4,5,6]], axis=1)), axis=1)*10) for i in range(len(states))]


[1.8094570476672747,
 1.8123752732339262,
 1.8786770158215405,
 1.85468155060473,
 1.8180035817997187,
 1.9337323459400422,
 2.0430723518128997,
 1.9134258543002518]

In [276]:
signal = np.cos(np.arange(0, 7*np.pi, 0.001))
python_res = pd.DataFrame(rainflow.extract_cycles(signal), columns=["Range", "Mean", "Count", "Start idx", "End idx"])
python_res.sort_values('Range', inplace=True, ignore_index=True)
dx = np.diff(signal)
tps = np.concatenate([[True], (dx[1:] * dx[:-1] < 0), [True]])
nrel_res = pd.DataFrame(rainflow_nrel(np.array(signal[tps])).transpose(), columns=['Range', 'Mean', 'GAR', 'Count', 'GAR-ZFLM'])
nrel_res.sort_values('Range', inplace=True, ignore_index=True)

In [277]:
python_res

,Range,Mean,Count,Start idx,End idx
0,2.0,-4.378278e-08,0.5,18850,21991
1,2.0,3.289809e-08,1.0,3142,6283
2,2.0,-4.896411e-08,0.5,15708,18850
3,2.0,-2.201340e-08,1.0,9425,12566
4,2.0,3.373109e-10,0.5,0,15708


In [278]:
nrel_res

,Range,Mean,GAR,Count,GAR-ZFLM
0,2.0,-4.378278e-08,2.0,0.5,2.0
1,2.0,3.289809e-08,2.0,1.0,2.0
2,2.0,-4.896411e-08,2.0,0.5,2.0
3,2.0,-2.201340e-08,2.0,1.0,2.0
4,2.0,3.373109e-10,2.0,0.5,2.0


In [1]:
refs = [load_ipc_reference(g, reference_dirs=['actuator_turb']) for g in range(140,147)]

NameError: name 'load_ipc_reference' is not defined

In [101]:
keys = ['oop blade root bending moment blade 1 [N]', 'oop blade root bending moment blade 2 [N]', 'oop blade root bending moment blade 3 [N]']
np.mean([[calc_del(np.array(r[key]), m=10) for r in refs] for key in keys])

14606528.679850237

In [110]:
keys = ['pitch blade 1 [deg]', 'pitch blade 2 [deg]', 'pitch blade 3 [deg]']
np.mean([[calc_del(np.array(r[key]), m=1) for r in refs] for key in keys])

0.3189590007631355

In [108]:
np.mean([r['reward'].sum() for r in refs])

615.9732283162311

In [64]:
develop = [calc_something_like_del(parsed['orig_states']['oop blade root bending moment blade 3 [N]'][0:i], texp=2, m=10) for i in range(10,2000)]

In [65]:
px.line(develop)

In [57]:
px.line(parsed['orig_states']['oop blade root bending moment blade 1 [N]'])